<a href="https://colab.research.google.com/github/UPstartDeveloper/DS-2.4-Advanced-Topics/blob/main/Notebooks/Computer_Vision/Plant_Vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diagnosing Crop Leaves

Source: in this notebook we will use the "New Plant Diseases Dataset", originally uploaded to [Kaggle](https://www.kaggle.com/vipoooool/new-plant-diseases-dataset) by Samir Bhattarai.


**MY PROCESS**: I started with the [instructions from video by Krish Naik](https://youtu.be/chQNuV9B-Rw) to do transfer learning with Inception V3, then went ahead with the following adjustments:

1. There was an imbalance in the number of labels represented in the `test/` data directory that originally came with the dataset. So instead I combined it all into the `train/` directory, and then split the data into training and testing sections using `ImageDataGenerator`.
2. The original notebook used Inception V3 - I decided against using transfer learning to start out with, so I could learn a little more about building a *Convolutional Neural Network* (CNN) in the beginning versions of this model.

## Let's Train on the GPU

## Import Packages

In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model, Sequential
# Pre-trained Models We May Use for Transfer learning
from tensorflow.keras.applications.inception_v3 import InceptionV3
# Preprocessing Data 
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
# Keras Callbacks
from tensorflow.keras.callbacks import (
    EarlyStopping, 
    ModelCheckpoint,
    TensorBoard,
)
# ease of handling data, and visualizing data
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
# Layers for the Model
from tensorflow import keras
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPooling2D,
    Dropout,
    Flatten,
    BatchNormalization
)
%matplotlib inline

## Get the Data 
The file paths to the data are below:

In [2]:
DATA_PATH = './new-plant-diseases-dataset'
TRAIN_PATH = f"{DATA_PATH}/train"
VAL_PATH = f"{DATA_PATH}/valid"
# The test directory was empty on my machine
# TEST_PATH = f"{DATA_PATH}/test"

## How Many Classes Are in the Dataset?

In [3]:
# get the number of output classes
NUM_CLASSES = len(glob(f"{TRAIN_PATH}/*"))  # 38
print(f"There are {NUM_CLASSES} classes in the dataset.")

There are 38 classes in the dataset.


## What Are the Dimensions of the Dataset?

In [4]:
# desired image dimensions
IMAGE_SIZE = [224, 224, 3]

## Examples of the Data

In [5]:
# TODO: print examples using imshow

# Defining the CNN Architecture
This model is an example of a CNN (*convolutional neural network*). 

We will be using the Sequential API in this implementation. The architecture is summarized below:

- 3 Convolutional Layers, 4 neurons each
- 1 Dense layer
- Optimizer Algorithm is Adam
- Drop Out Rate is 20%

### Utility Functions to Make Adding layers Easier

In [6]:
def add_conv_layer(model, layer_size, needs_input, input_shape, kernel_size=None, pool_size=None):
    """Add a Keras convolutional layer to the model, along with MaxPooling.
       Will specify input shape as well if needed.
       
       Parameters:
       model(Model): Neural network in Keras
       layer_size(int): number of neurons to go in layer
       need_input(bool): signals if the convolutional layer needs to specify
                         the dimensions of the input
       input_shape(tuple or list): 1D vector of what the dataset dimensions are
       kernel_size(tuple): specifies a square matrix to use for kernel dimensions
       pool_size(tuple): specifies a square matrix to use in pooling
       
       Returns: None
       
    """
    # set kernel and pool size
    if kernel_size is None:
        kernel_size = (3, 3)
    if pool_size is None:
        pool_size = (2, 2)
    # specify input dimension for 1st conv layer
    if needs_input is True:
        conv_layer = Conv2D(layer_size,
                            kernel_size=kernel_size,
                            activation='relu',
                            input_shape=input_shape)

    else:
        # otherwise all other convolutional layers don't need it
        conv_layer = Conv2D(layer_size,
                            kernel_size=kernel_size,
                            activation='relu')
    # add Convolutional layer
    model.add(conv_layer)  
    # add MaxPooling layer
    model.add(MaxPooling2D(pool_size=pool_size))  # no learning params
    return None
                  
    
def add_dense_layer(model, layer_size, is_output, drop_rate):
    """Add a multi-layer perceptron to the model
       Will specify 'softmax' for the final layer.
       
       Parameters:
       model(Model): Neural network in Keras
       layer_size(int): number of neurons to go in layer
       is_output(bdool): signals if the MLP is the last layer
       drop_rate(float): percentage of connections in Dense layer
                       to cut off
       
       Returns: None
       
    """
    # specify activation function
    activation = 'relu' if is_output is False else 'softmax'
    # add MLP
    model.add(Dense(layer_size, activation=activation)) 
    # Add Dropout layer and Batch Normalization
    if is_output is False:
        model.add(BatchNormalization())
        model.add(Dropout(drop_rate))
    return None


def compile_model(model, optimizer=None):
    """Compile the neural network.
    
       Parameter:
       model(keras.Sequential or keras.Model): the model object
       optimizer(str): specfies the algoritm used to minimize loss
       
       Returns: None
       
    """
    # set the optimizer
    if optimizer is None:
        optimizer = 'adam'
    # compile the model
    model.compile(
        loss='categorical_crossentropy', optimizer=optimizer,
        metrics=[
          'accuracy', tf.keras.metrics.Precision(),
          tf.keras.metrics.Recall()]
    )
    return None

def define_model(units, conv_layers, dense_layers, dropout, input_shape):
    """Define a CNN + MLP model in Keras.
    
       Parameters:
       units(int): number of neurons to go in a layer
       conv_layers(int): number of convolutional layers
       dense_layers(int): number of MLP
       dropout(float): percentage of connections in the Dense layer(s) to cut 
        input_shape(tuple or list): 1D vector of what the dataset dimensions are
                       
       Returns: tf.keras.Sequential: the neural network to train
    
    """
    # Instaniate model
    model = Sequential()
    # Add CNN layers
    add_conv_layer(model, units, True, input_shape)
    for l in range(conv_layers - 1):
        # add convolutional layers that come after the 1st
        add_conv_layer(model, units, False, input_shape)
    # Flatten the data
    model.add(Flatten())
    # Add MLP Layers
    for l in range(dense_layers - 1):
        add_dense_layer(model, units, False, dropout)
    # add final MLP, for output
    add_dense_layer(model, NUM_CLASSES, True, dropout)
    # Compile Model
    compile_model(model)
    return model

### Instantiation of the Model

In [7]:
# build the model
conv_layers = 3
dense_layers = 1
layer_size = 4

# Define Model
model = define_model(layer_size, conv_layers, dense_layers, 0.2, IMAGE_SIZE)

## Summarize the Model

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 4)       112       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 4)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 4)       148       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 4)         148       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 4)         0         
_________________________________________________________________
flatten (Flatten)            (None, 2704)              0

## Set the Training Parameters

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

### Use Data Generators
Due to the size of the dataset (87K images), this step will make the training process more efficient.

We scale all the data, and data augmentation is only on the training data (to make the model more robust).

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255, shear_range=0.2,
    zoom_range=0.2, horizontal_flip = True,
    validation_split=0.2  # set aside some data for validation
  )

test_datagen = ImageDataGenerator(rescale = 1./255)

### Load the Train, Test, and Validation Sets
We scale all the data, and data augmentation is only on the training data (to make the model more robust).

In [11]:
# Make sure you provide the same target size as initialized for the image size
image_width, image_height = IMAGE_SIZE[0], IMAGE_SIZE[1]
training_set = train_datagen.flow_from_directory(
    TRAIN_PATH, target_size=(image_width, image_height),
    batch_size=80, class_mode='categorical',
    subset='training'
  )

Found 56251 images belonging to 38 classes.


In [12]:
# use the rest of the train/ dir for validation data
image_width, image_height = IMAGE_SIZE[0], IMAGE_SIZE[1]
val_set = train_datagen.flow_from_directory(
    TRAIN_PATH, target_size=(image_width, image_height),
    batch_size=80, class_mode='categorical',
    subset='validation'
  )

Found 14044 images belonging to 38 classes.


In [13]:
# use the valid/ dir for testing data
image_width, image_height = IMAGE_SIZE[0], IMAGE_SIZE[1]
test_set = test_datagen.flow_from_directory(
    VAL_PATH, target_size=(image_width, image_height),
    batch_size=80, class_mode='categorical' 
)

Found 18050 images belonging to 38 classes.


### Using Keras Callbacks

In [14]:
# use TensorBoard to monitor training
# name the model for TensorBoard
NAME = (
    f'{3}-conv-{4}-nodes' +
    f'-{1}-dense_layers'
)
# Instantiate TensorBoard to visualize model performance
tensorboard = TensorBoard(log_dir=f'./Graph/{NAME}')

# using ModelCheckpoint, for convenience of coming back to train more later
checkpoint_filepath = './tmp/checkpoint'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,  # we'll need to remake the architecture again
    monitor='accuracy',
    mode='max',
    save_freq='epoch',  # save every epoch
    save_best_only=True)

## Train the Model

In [15]:
while True:  # preventing runtime timeout
    history = model.fit(
      x=training_set,  # pass in the generator to determine x, y, and batch_size
      epochs=10, validation_data=val_set,
      steps_per_epoch=len(training_set),
      validation_steps=len(val_set),
      callbacks=[
        tensorboard, model_checkpoint_callback
      ]
    )
    break

Epoch 1/10
704/704 [==============================] - 886s 1s/step - loss: 2.6274 - accuracy: 0.2811 - val_loss: 1.6224 - val_accuracy: 0.5377
Epoch 2/10
704/704 [==============================] - 839s 1s/step - loss: 1.4842 - accuracy: 0.5688 - val_loss: 1.4973 - val_accuracy: 0.5666
Epoch 3/10
704/704 [==============================] - 831s 1s/step - loss: 1.3800 - accuracy: 0.5961 - val_loss: 1.3979 - val_accuracy: 0.5911
Epoch 4/10
704/704 [==============================] - 852s 1s/step - loss: 1.2966 - accuracy: 0.6211 - val_loss: 1.3611 - val_accuracy: 0.6012
Epoch 5/10
704/704 [==============================] - 888s 1s/step - loss: 1.2539 - accuracy: 0.6285 - val_loss: 1.3147 - val_accuracy: 0.6151
Epoch 6/10
704/704 [==============================] - 1383s 2s/step - loss: 1.2010 - accuracy: 0.6446 - val_loss: 1.2909 - val_accuracy: 0.6179
Epoch 7/10
704/704 [==============================] - 1776s 3s/step - loss: 1.1901 - accuracy: 0.6478 - val_loss: 1.2373 - val_accuracy: 0.63

## Evaluate the Model

In [16]:
def plot_metric(history, metric):
    """Plots the model performance metric for training
       testing data using Matplotlib.
       
       Parameters
       history(History): contains data on model's metrics over the
                         course of the training
       metric(str): the name of the metric. Will be one of 
                    the keys in the History object, e.g. 'loss'
       
       Returns: None
       
    """
    # plot for training data
    plt.plot(history.history[metric])
    # plot for testing data
    val_form = f'val_{metric}'
    plt.plot(history.history[val_form])
    # add meta-info about graph - title, labels, legend
    plt.title(f'Model {metric}')
    plt.ylabel(metric)
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    # show the graph
    plt.show()
    return None


In [17]:
# TODO: plot validation accuracy 

In [18]:
# TODO plot validation loss

## Export the Model

In [20]:
def save_model(model, weights_file, architecture_file):
    """Save the model weights and architecture.
    
       Parameters: 
       model(Model): keras Model object being saved
       weights_file(str): name of the Hadoop file where
                          weights will be saved
       architecture_file(str): name of the JSON file where 
                               model architecture is to be
                               saved
                               
       Returns: None
       
    """
    # Save the weights
    model.save_weights(f'{weights_file}.h5')
    # Save the architecture
    with open(f'{architecture_file}.json', 'w') as f:
        f.write(model.to_json())
    return None

save_model(model, "simple_cnn_weights", "simple_cnn_architecture")

## Final Conclusions

## Next Steps

1. Use [TensorBoard](https://www.ecosia.org/search?q=tensorboard&addon=chrome&addonversion=3.2.0&method=topbar) and [Hyperas](https://github.com/maxpumperla/hyperas/issues) to experiment with what the best architecture and hyperparameters to experiment with what the best hyperparameters are for the model.
2. Check for class imbalances, and if there are try to upsample the minority classes (or adjust the class weights) in order to address the issue.
3. Try using pre-trained weights from a popular CNN like ResNet or Inception, and see if we get a better performing model.